In [22]:
import pandas as pd

df = pd.read_excel('readings.xlsx', header=1)

In [26]:
df.fillna("NULL", inplace=True)
df.drop("Unnamed: 8", axis=1, inplace=True)

In [27]:
df.head()

,title,ailmentID,type,assign_range,low_range,high_range,isGraph,unit,2,3,4,5,6,7,8,9,10
0,Weight before dialysis,1,int,yes,0,0,1,Kgs,डायलिसिस से पहले वजन,ડાયાલિસિસ પહેલા નો વજન,ಡಯಾಲಿಸಿಸ್ ಗಿಂತ ಮೊದಲಿನ ತೂಕ,ডায়েলাইছিছৰ আগতে ওজন,डायलिसिस करण्यापूर्वी वजन,டயாலிசிஸுக்கு முன் எடை,NULL,NULL,NULL
1,Weight after dialysis,1,int,yes,0,0,1,Kgs,डायलिसिस के बाद वजन,ડાયાલિસિસ પછી નો વજન,ಡಯಾಲಿಸಿಸನ ನಂತರದ ತೂಕ,ডায়েলাইছিছৰ পিছত ওজন,डायलिसिस नंतर वजन,டயாலிசிஸ் பிறகு எடை,NULL,NULL,NULL
2,Temperature before dialysis,1,int,yes,0,0,1,°F,डायलिसिस से पहले तापमान,ડાયાલિસિસ પહેલા નું તાપમાન,ಡಯಾಲಿಸಿಸ್ ನ ಮೊದಲಿನ ದೇಹದ ಉಷ್ಣತೆ,ডায়েলাইছিছৰ আগতে উষ্ণতা,डायलिसिस करण्यापूर्वी तापमान,டயாலிசிஸுக்கு முன் வெப்பநிலை,NULL,NULL,NULL
3,Temperature after dialysis,1,int,yes,0,0,1,°F,डायलिसिस के बाद तापमान,ડાયાલિસિસ પછી નું તાપમાન,ಡಯಾಲಿಸಿಸ್ ನ ನಂತರದ ದೇಹದ ಉಷ್ಣತೆ,ডায়েলাইছিছৰ পিছত উষ্ণতা,डायलिसिस नंतर तापमान,டயாலிசிஸ் பிறகு வெப்பநிலை,NULL,NULL,NULL
4,Systolic (Upper reading) Blood pressure after ...,1,int,yes,0,0,1,mmHg,सिस्टोलिक (ऊपर की रीडिंग) रक्तचाप डायलिसिस के ...,ડાયાલિસિસ પછી સિસ્ટોલિક બ્લડ પ્રેશર (ઉપર ની રી...,ಡಯಾಲಿಸಿಸನ ನಂತರದ ಸಿಸ್ಟೋಲಿಕ್ ರಕ್ತದೊತ್ತಡ,ডায়েলাইছিছৰ পিছত চিষ্টোলিক (শীৰ্ষ পঢ়া) ৰক্তচাপ,डायलिसिस नंतर सिस्टोलिक (टॉप रीडिंग) रक्तदाब,டயாலிசிஸுக்குப் பிறகு சிஸ்டாலிக் (மேல் வாசிப்ப...,NULL,NULL,NULL


## FOR DAILY READINGS

In [32]:
import os
import mysql.connector
from dotenv import load_dotenv

DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "password"
DB_DATABASE = "kifayti"

# Connect to MySQL database
try:
    connection = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_DATABASE
    )
    if connection.is_connected():
        print("Connected to MySQL database")
        # Perform database operations here

        cursor = connection.cursor()
        for i, row in df.iterrows():
            try:
                cursor = connection.cursor()
                query = '''
                    INSERT INTO `daily_readings` 
                    (`id`, `title`, `ailmentID`, `assign_range`, `type`, `low_range`, `high_range`, 
                    `isGraph`, `showUser`, `unit`, `priority_type`, `alertTextDoc`, `createdAt`, `updatedAt`) 
                    VALUES (NULL, %s, %s, %s, %s, %s, %s, %s, "0", %s, NULL, NULL, '2024-04-20 07:40:07.000000', '2024-04-20 07:40:07.000000')
                '''
                values = (row["title"], row["ailmentID "], row["assign_range"], row["type"], row["low_range"], row["high_range"], row["isGraph"], row["unit"])
                cursor.execute(query, values)
                inserted_id = cursor.lastrowid
                print(f"Inserted ID: {inserted_id}")
                for j in range(2, df.columns[-1]+1):
                    if row[j] != "NULL":
                        query = '''
                            INSERT INTO `daily_readings_translations` 
                            (`id`, `dr_id`, `language_id`, `title`,`createdAt`, `updatedAt`) 
                            VALUES (NULL, %s, %s,%s, '2024-04-20 07:40:07.000000', '2024-04-20 07:40:07.000000')
                        '''
                        values = (inserted_id, j ,row[j])
                        cursor.execute(query, values)
                connection.commit()
                print("Data inserted successfully")
            except mysql.connector.Error as e:
                print(f"Error inserting data: {e}")


except mysql.connector.Error as e:
    print(f"Error connecting to MySQL database: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        connection.close()
        print("MySQL connection is closed")


Connected to MySQL database
Inserted ID: 1
Data inserted successfully
Inserted ID: 2
Data inserted successfully
Inserted ID: 3
Data inserted successfully
Inserted ID: 4
Data inserted successfully
Inserted ID: 5
Data inserted successfully
Inserted ID: 6
Data inserted successfully
Inserted ID: 7
Data inserted successfully
Inserted ID: 8
Data inserted successfully
Inserted ID: 9
Data inserted successfully
Inserted ID: 10
Data inserted successfully
Inserted ID: 11
Data inserted successfully
Inserted ID: 12
Data inserted successfully
Inserted ID: 13
Data inserted successfully
Inserted ID: 14
Data inserted successfully
Inserted ID: 15
Data inserted successfully
Inserted ID: 16
Data inserted successfully
Inserted ID: 17
Data inserted successfully
Inserted ID: 18
Data inserted successfully
Inserted ID: 19
Data inserted successfully
Inserted ID: 20
Data inserted successfully
Inserted ID: 21
Data inserted successfully
Inserted ID: 22
Data inserted successfully
Inserted ID: 23
Data inserted succe

## FOR DIALYSIS READINGS

In [ ]:
import os
import mysql.connector
from dotenv import load_dotenv

DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "password"
DB_DATABASE = "kifayti"

# Connect to MySQL database
try:
    connection = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_DATABASE
    )
    if connection.is_connected():
        print("Connected to MySQL database")
        # Perform database operations here

        cursor = connection.cursor()
        for i, row in df.iterrows():
            try:
                cursor = connection.cursor()
                query = '''
                    INSERT INTO `dialysis_readings` 
                    (`id`, `title`, `ailmentID`, `assign_range`, `type`, `low_range`, `high_range`, 
                    `isGraph`, `showUser`, `unit`, `priority_type`, `alertTextDoc`, `createdAt`, `updatedAt`) 
                    VALUES (NULL, %s, %s, %s, %s, %s, %s, %s, "0", %s, NULL, NULL, '2024-04-20 07:40:07.000000', '2024-04-20 07:40:07.000000')
                '''
                values = (row["title"], row["ailmentID "], row["assign_range"], row["type"], row["low_range"], row["high_range"], row["isGraph"], row["unit"])
                cursor.execute(query, values)
                inserted_id = cursor.lastrowid
                print(f"Inserted ID: {inserted_id}")
                for j in range(2, df.columns[-1]+1):
                    if row[j] != "NULL":
                        query = '''
                            INSERT INTO `dialysis_readings_translations` 
                            (`id`, `dr_id`, `language_id`, `title`,`createdAt`, `updatedAt`) 
                            VALUES (NULL, %s, %s,%s, '2024-04-20 07:40:07.000000', '2024-04-20 07:40:07.000000')
                        '''
                        values = (inserted_id, j ,row[j])
                        cursor.execute(query, values)
                connection.commit()
                print("Data inserted successfully")
            except mysql.connector.Error as e:
                print(f"Error inserting data: {e}")


except mysql.connector.Error as e:
    print(f"Error connecting to MySQL database: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        connection.close()
        print("MySQL connection is closed")
